In [1]:
import pandas as pd
import numpy as np
import nltk
from gc import collect
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
import re
import time
%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cerdgio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
start = time.time()

PATH = './input/'
nrows = None
SEED = 999

In [3]:
data_train = pd.read_csv('{PATH}train.csv'.format(PATH = PATH), nrows = nrows, index_col='item_id')
print(data_train.shape)
train_index = data_train.shape[0]
data_train = data_train.append(pd.read_csv('{PATH}test.csv'.format(PATH = PATH), 
                                           nrows = nrows, index_col='item_id'))
print(data_train.shape)

(1503424, 17)


/home/cerdgio/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


(2011862, 17)


In [4]:
data_train['activation_date'] = pd.to_datetime(data_train['activation_date'])
data_train['day'] = data_train['activation_date'].dt.day
data_train['weekday'] = data_train['activation_date'].dt.dayofweek
data_train['region_city'] = data_train['region']+'_'+data_train['city']
data_train['parent_category'] = data_train['parent_category_name']+'_'+data_train['category_name']
data_train['logprice'] = np.log(data_train['price']+.000001)
data_train['subtarget'] = data_train['deal_probability'].astype(bool)


In [5]:
data_train['pirce_qbin'] = pd.qcut(data_train['price'], np.linspace(0, 1., 21), duplicates='drop')
data_train['pirce_bin'] = pd.cut(data_train['price'], [0, 50, 100, 200, 300, 500, 700, 1000, 2000, 
                                                       3000, 5000, 7000, 10000, 20000, 
                                                       30000, 50000, 70000, 100000, 200000, 
                                                       300000, 500000, 700000, 1000000, np.Inf])
data_train['logpirce_qbin'] = pd.qcut(data_train['logprice'], np.linspace(0, 1., 21), duplicates='drop')

In [6]:
data_train['item_seq_number_bin'] = pd.cut(data_train['item_seq_number'], [0,1,2,3,4,5,6,7,8,9,10,12,14,16,18,20,25,30,35,40,45,50,
                                       60,70,80,90,100,200,300,400,500,600,700,800,900,1000,2000,3000,
                                       4000,5000,10000, np.Inf])
data_train['item_seq_number_qbin'] = pd.qcut(data_train['item_seq_number'], np.linspace(0, 1., 21), 
                                             duplicates='drop', )

In [7]:
cat_cols = \
['region','city','region_city','parent_category_name','day', 'weekday', 'user_type','pirce_bin', 'pirce_qbin',
 'logpirce_qbin', 'item_seq_number_bin', 'item_seq_number_qbin',
 'category_name','parent_category','image_top_1', 'item_seq_number']
drop_cols = ['image']

In [8]:
for col in data_train.columns:
    if (data_train[col].dtypes=='object'):
        data_train[col].fillna('-999', inplace=True)
    elif (data_train[col].dtype.name=='category'):
        data_train[col] = data_train[col].astype(str)
        data_train[col].fillna(-999, inplace=True)
    else:
        data_train[col].fillna(-999, inplace=True)

In [9]:
def add_col(col_names, col):
    for i in col_names:
        data_train[i+'_'+col] = data_train[i].astype(str).values+'_'+data_train[col].astype(str).values
        cat_cols.append(i+'_'+col)

add_col(['region','city','region_city','parent_category_name','user_type','pirce_bin', 'pirce_qbin',
 'logpirce_qbin', 'item_seq_number_bin', 'item_seq_number_qbin',
 'category_name','parent_category','image_top_1', 'item_seq_number'], 'weekday')
add_col(['region','city','region_city','parent_category_name','pirce_bin', 'pirce_qbin',
 'logpirce_qbin', 'item_seq_number_bin', 'item_seq_number_qbin',
 'category_name','parent_category','image_top_1', 'item_seq_number'],'user_type')
add_col(['region','city','region_city','pirce_bin', 'pirce_qbin',
 'logpirce_qbin', 'item_seq_number_bin', 'item_seq_number_qbin','parent_category','image_top_1', 'item_seq_number'],
        'category_name')
add_col(['pirce_bin', 'pirce_qbin',
 'logpirce_qbin', 'item_seq_number_bin', 'item_seq_number_qbin',
         'item_seq_number'],
        'city')

In [10]:
from utils import TargetEncoding
from sklearn.model_selection import StratifiedKFold

In [11]:
te = TargetEncoding()

In [12]:
data_test = data_train[train_index:].copy()
data_test.drop(['deal_probability'], axis = 1, inplace=True)
print(data_test.shape)
data_train = data_train[:train_index].copy()
print(data_train.shape)

(508438, 71)
(1503424, 72)


In [13]:
skf = StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
y = data_train['subtarget']
for train_index, test_index in skf.split(data_train, y):
    X_train, X_test = data_train.iloc[train_index], data_train.iloc[test_index]
    y_train, _ = y[train_index], y[test_index]
    for col in cat_cols:
        if col+'_te_bin' in data_train.columns:
            pass
        else:
            data_train[col+'_te_bin'] = 999
        te.fit(X_train[col], y_train)
        #print(te.transform(X_test[col]).shape, X_test[col].shape, test_index.shape, train_index.shape)
        data_train.loc[data_train.index[test_index], col+'_te_bin'] = te.transform(X_test[col])
        
        if col+'_te' in data_train.columns:
            pass
        else:
            data_train[col+'_te'] = 999
        te.fit(X_train[col], data_train.loc[data_train.index[train_index], 'deal_probability'])
        #print(te.transform(X_test[col]).shape, X_test[col].shape, test_index.shape, train_index.shape)
        data_train.loc[data_train.index[test_index], col+'_te'] = te.transform(X_test[col])
################## alpha 200    
        if col+'_te_bin' in data_train.columns:
            pass
        else:
            data_train[col+'_te_bin_200'] = 999
        te.fit(X_train[col], y_train, alpha=200)
        #print(te.transform(X_test[col]).shape, X_test[col].shape, test_index.shape, train_index.shape)
        data_train.loc[data_train.index[test_index], col+'_te_bin_200'] = te.transform(X_test[col])
        
        if col+'_te' in data_train.columns:
            pass
        else:
            data_train[col+'_te_200'] = 999
        te.fit(X_train[col], data_train.loc[data_train.index[train_index], 'deal_probability'], alpha=200)
        #print(te.transform(X_test[col]).shape, X_test[col].shape, test_index.shape, train_index.shape)
        data_train.loc[data_train.index[test_index], col+'_te_200'] = te.transform(X_test[col])
################## alpha 1000        
        if col+'_te_bin' in data_train.columns:
            pass
        else:
            data_train[col+'_te_bin_100'] = 999
        te.fit(X_train[col], y_train, alpha = 1000)
        #print(te.transform(X_test[col]).shape, X_test[col].shape, test_index.shape, train_index.shape)
        data_train.loc[data_train.index[test_index], col+'_te_bin_100'] = te.transform(X_test[col])
        
        if col+'_te' in data_train.columns:
            pass
        else:
            data_train[col+'_te_100'] = 999
        te.fit(X_train[col], data_train.loc[data_train.index[train_index], 'deal_probability'], alpha = 1000)
        #print(te.transform(X_test[col]).shape, X_test[col].shape, test_index.shape, train_index.shape)
        data_train.loc[data_train.index[test_index], col+'_te_100'] = te.transform(X_test[col])

In [14]:
for col in cat_cols:
    
        te.fit(data_train.loc[:,col],  data_train['subtarget'])
        #print(te.transform(X_test[col]).shape, X_test[col].shape, test_index.shape, train_index.shape)
        data_test[col+'_te_bin'] = te.transform(data_test[col])
    
        te.fit(data_train.loc[:,col], data_train['deal_probability'])
        #print(te.transform(X_test[col]).shape, X_test[col].shape, test_index.shape, train_index.shape)
        data_test[col+'_te'] = te.transform(data_test[col])
################## alpha 200    

        te.fit(data_train.loc[:,col],  data_train['subtarget'], alpha=200)
        #print(te.transform(X_test[col]).shape, X_test[col].shape, test_index.shape, train_index.shape)
        data_test[col+'_te_bin_200'] = te.transform(data_test[col])
        
        te.fit(data_train.loc[:,col], data_train['deal_probability'], alpha=200)
        #print(te.transform(X_test[col]).shape, X_test[col].shape, test_index.shape, train_index.shape)
        data_test[col+'_te_200'] = te.transform(data_test[col])
################## alpha 1000        

        te.fit(data_train.loc[:,col],  data_train['subtarget'], alpha = 1000)
        #print(te.transform(X_test[col]).shape, X_test[col].shape, test_index.shape, train_index.shape)
        data_test[col+'_te_bin_100'] = te.transform(data_test[col])
        
        te.fit(data_train.loc[:,col], data_train['deal_probability'], alpha = 1000)
        #print(te.transform(X_test[col]).shape, X_test[col].shape, test_index.shape, train_index.shape)
        data_test[col+'_te_100'] = te.transform(data_test[col])

In [15]:
textfeats = ["description", "title", "param_1"]

def creat_str_features(merge, textfeats):
    from nltk.corpus import stopwords
    import string
    stopwords_en = {x: 1 for x in stopwords.words('english')}
    stopwords = {x: 1 for x in stopwords.words('russian')}
    non_alphanums = re.compile(u'[^A-Za-z0-9]+')
    non_alphanumpunct = re.compile(u'[^A-Za-z0-9\.?!,; \(\)\[\]\'\"\$]+')
    RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])
    
    def sum_numbers(desc):
        if not isinstance(desc, str):
            return 0
        try:
            return sum([to_number(s) for s in desc.split()])
        except:
            return 0
    
    for cols in textfeats:
        merge[cols] = merge[cols].astype(str) 
        merge[cols] = merge[cols].astype(str).fillna('missing') # FILL NA
        merge[cols] = merge[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
        merge[cols + '_num_stopwords'] = merge[cols].apply(lambda x: len([w for w in x.split() if w in stopwords])) # Count number of Stopwords
        merge[cols + '_num_stopwords_en'] = merge[cols].apply(lambda x: len([w for w in x.split() if w in stopwords_en])) # Count number of Stopwords
        merge[cols + '_num_punctuations'] = merge[cols].apply(lambda comment: (comment.count(RE_PUNCTUATION))) # Count number of Punctuations
        merge[cols + '_num_alphabets'] = merge[cols].apply(lambda comment: (comment.count(r'[a-zA-Z]'))) # Count number of Alphabets
        merge[cols + '_num_alphanumeric'] = merge[cols].apply(lambda comment: (comment.count(r'[A-Za-z0-9]'))) # Count number of AlphaNumeric
        merge[cols + '_num_digits'] = merge[cols].apply(lambda comment: (comment.count('[0-9]'))) # Count number of Digits
        merge[cols + '_num_letters'] = merge[cols].apply(lambda comment: len(comment)) # Count number of Letters
        merge[cols + '_num_words'] = merge[cols].apply(lambda comment: len(comment.split())) # Count number of Words
        merge[cols + '_num_unique_words'] = merge[cols].apply(lambda comment: len(set(w for w in comment.split())))
        merge[cols + '_words_vs_unique'] = merge[cols+'_num_unique_words'] / merge[cols+'_num_words'] # Count Unique Words
        merge[cols + '_letters_per_word'] = merge[cols+'_num_letters'] / merge[cols+'_num_words'] # Letters per Word
        merge[cols + '_punctuations_by_letters'] = merge[cols+'_num_punctuations'] / merge[cols+'_num_letters'] # Punctuations by Letters
        merge[cols + '_punctuations_by_words'] = merge[cols+'_num_punctuations'] / merge[cols+'_num_words'] # Punctuations by Words
        merge[cols + '_digits_by_letters'] = merge[cols+'_num_digits'] / merge[cols+'_num_letters'] # Digits by Letters
        merge[cols + '_alphanumeric_by_letters'] = merge[cols+'_num_alphanumeric'] / merge[cols+'_num_letters'] # AlphaNumeric by Letters
        merge[cols + '_alphabets_by_letters'] = merge[cols+'_num_alphabets'] / merge[cols+'_num_letters'] # Alphabets by Letters
        merge[cols + '_stopwords_by_letters'] = merge[cols+'_num_stopwords'] / merge[cols+'_num_letters'] # Stopwords by Letters
        merge[cols + '_stopwords_by_words'] = merge[cols+'_num_stopwords'] / merge[cols+'_num_words'] # Stopwords by Letters
        merge[cols + '_stopwords_by_letters_en'] = merge[cols+'_num_stopwords_en'] / merge[cols+'_num_letters'] # Stopwords by Letters
        merge[cols + '_stopwords_by_words_en'] = merge[cols+'_num_stopwords_en'] / merge[cols+'_num_words'] # Stopwords by Letters    
        merge[cols + '_mean'] = merge[cols].apply(lambda x: 0 if len(x) == 0 else float(len(x.split())) / len(x)) * 10 # Mean
        merge[cols + '_num_sum'] = merge[cols].apply(sum_numbers) 

    # Extra Feature Engineering
    merge['title_desc_len_ratio'] = merge['title_num_letters']/merge['description_num_letters']
    merge['title_param1_len_ratio'] = merge['title_num_letters']/merge['param_1_num_letters']
    merge['param_1_copy_desc_len_ratio'] = merge['param_1_num_letters']/merge['description_num_letters']
    
    return merge

In [16]:
data_train = creat_str_features(data_train, textfeats)
data_test = creat_str_features(data_test, textfeats)
data_train.drop(drop_cols, axis=1, inplace=True)
data_test.drop(drop_cols, axis=1, inplace=True)

In [17]:
data_train.describe()

,deal_probability,image_top_1,item_seq_number,price,day,weekday,logprice,region_te_bin,region_te,region_te_bin_200,...,param_1_alphabets_by_letters,param_1_stopwords_by_letters,param_1_stopwords_by_words,param_1_stopwords_by_letters_en,param_1_stopwords_by_words_en,param_1_mean,param_1_num_sum,title_desc_len_ratio,title_param1_len_ratio,param_1_copy_desc_len_ratio
count,1.503424e+06,1.503424e+06,1.503424e+06,1.503424e+06,1.503424e+06,1.503424e+06,1.503424e+06,1.503424e+06,1.503424e+06,1.503424e+06,...,1503424.0,1.503424e+06,1.503424e+06,1503424.0,1503424.0,1.503424e+06,1503424.0,1.503424e+06,1.503424e+06,1.503424e+06
mean,1.391306e-01,1.074114e+03,7.436740e+02,2.986692e+05,2.155644e+01,2.942614e+00,-4.935659e+01,3.517341e-01,1.391304e-01,3.517442e-01,...,0.0,2.705155e-02,1.527001e-01,0.0,0.0,1.582316e+00,0.0,6.564701e-01,2.294522e+00,4.475241e-01
std,2.600785e-01,1.104164e+03,5.572522e+03,6.496484e+07,4.045831e+00,2.031382e+00,2.330120e+02,3.512749e-02,9.269730e-03,3.495678e-02,...,0.0,3.926823e-02,2.176477e-01,0.0,0.0,4.329003e-01,0.0,1.536704e+00,1.878767e+00,9.408213e-01
min,0.000000e+00,-9.990000e+02,1.000000e+00,-9.990000e+02,1.000000e+00,0.000000e+00,-9.990000e+02,2.927366e-01,1.199856e-01,2.929241e-01,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,6.250000e-01,0.0,9.730782e-04,3.448276e-02,8.382230e-04
25%,0.000000e+00,1.250000e+02,9.000000e+00,4.000000e+02,1.800000e+01,1.000000e+00,5.991465e+00,3.391020e-01,1.343862e-01,3.391534e-01,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1.428571e+00,0.0,1.000000e-01,1.000000e+00,5.128205e-02
50%,0.000000e+00,9.600000e+02,2.900000e+01,1.000000e+03,2.200000e+01,3.000000e+00,6.907755e+00,3.508611e-01,1.425650e-01,3.508644e-01,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1.538462e+00,0.0,2.040816e-01,1.700000e+00,1.320755e-01
75%,1.508700e-01,2.078000e+03,8.800000e+01,6.000000e+03,2.500000e+01,5.000000e+00,8.699515e+00,3.705866e-01,1.462397e-01,3.704588e-01,...,0.0,7.142857e-02,3.333333e-01,0.0,0.0,1.818182e+00,0.0,4.375000e-01,3.000000e+00,3.076923e-01
max,1.000000e+00,3.066000e+03,2.044290e+05,7.950101e+10,3.100000e+01,6.000000e+00,2.509904e+01,4.241756e-01,1.562561e-01,4.236287e-01,...,0.0,1.250000e-01,5.000000e-01,0.0,0.0,5.000000e+00,0.0,5.000000e+01,2.500000e+01,2.900000e+01


In [18]:
data_test.describe()

,image_top_1,item_seq_number,price,day,weekday,logprice,region_te_bin,region_te,region_te_bin_200,region_te_200,...,param_1_alphabets_by_letters,param_1_stopwords_by_letters,param_1_stopwords_by_words,param_1_stopwords_by_letters_en,param_1_stopwords_by_words_en,param_1_mean,param_1_num_sum,title_desc_len_ratio,title_param1_len_ratio,param_1_copy_desc_len_ratio
count,508438.000000,508438.000000,5.084380e+05,508438.000000,508438.000000,508438.000000,508438.000000,508438.000000,508438.000000,508438.000000,...,508438.0,508438.000000,508438.000000,508438.0,508438.0,508438.000000,508438.0,508438.000000,508438.000000,508438.000000
mean,1105.465481,825.132150,2.629264e+05,14.985650,2.763963,-52.661586,0.351119,0.139072,0.351126,0.139072,...,0.0,0.026136,0.146968,0.0,0.0,1.585041,0.0,0.320509,2.361140,0.220038
std,1118.632459,5646.868618,5.200803e+06,2.092457,1.951071,239.435197,0.034839,0.009297,0.034726,0.009262,...,0.0,0.038883,0.214309,0.0,0.0,0.442198,0.0,0.572874,1.928929,0.396500
min,-999.000000,1.000000,-9.990000e+02,12.000000,0.000000,-999.000000,0.294829,0.120357,0.294950,0.120427,...,0.0,0.000000,0.000000,0.0,0.0,0.625000,0.0,0.001683,0.071429,0.000651
25%,231.000000,8.000000,4.000000e+02,13.000000,1.000000,5.991465,0.339341,0.135480,0.339375,0.135496,...,0.0,0.000000,0.000000,0.0,0.0,1.428571,0.0,0.093750,1.041667,0.046053
50%,1046.000000,30.000000,1.200000e+03,15.000000,3.000000,7.090077,0.347971,0.142602,0.347992,0.142593,...,0.0,0.000000,0.000000,0.0,0.0,1.538462,0.0,0.184211,1.750000,0.117117
75%,2188.000000,94.000000,7.500000e+03,17.000000,4.000000,8.922658,0.364654,0.146608,0.364625,0.146587,...,0.0,0.062500,0.333333,0.0,0.0,1.818182,0.0,0.357143,3.142857,0.250000
max,3066.000000,205064.000000,3.000060e+09,20.000000,6.000000,21.821898,0.422564,0.155921,0.422204,0.155807,...,0.0,0.125000,0.500000,0.0,0.0,5.000000,0.0,48.000000,25.000000,25.000000


In [19]:
data_train.to_csv(f'{PATH}train_features.csv')
data_test.to_csv(f'{PATH}test_features.csv')

In [20]:
collect()

7

In [21]:
time.strftime('%H:%m')

'03:06'